In [1]:
import requests
from lxml import html
from robots import Robot

In [2]:
def get_parsed_page(url):
    return html.fromstring(requests.get(url).content)

In [14]:
robot = Robot()
url_doc = {}
to_do = ['https://search.crossref.org/']

In [15]:
MAX_NUM = 5

In [16]:
for url in to_do:
    
#     print(url)
    
    if len(url_doc) >= MAX_NUM:
        break
        
    if url in url_doc or not robot.is_allowed(url):
        continue
        
    parsed_page = get_parsed_page(url)
    url_doc[url] = parsed_page
    
    post_urls = parsed_page.xpath('//a/@href')
    for post_url in post_urls:
        if len(post_url) == 0:
            continue
        if post_url[0] == '/':
            post_url = url + (post_url[1:] if url[-1] == '/' else post_url)
        if post_url not in url_doc and robot.is_allowed(post_url):
            to_do.append(post_url)

https://search.crossref.org/
https://search.crossref.org/funding
https://search.crossref.org/references
https://search.crossref.org/help/status
http://api.crossref.org
https://search.crossref.org/help/search
